# Week 03 – Authentication and Access Control  
## Networks and Systems Security



---

##  Section 1 – Password Strength Analysis

**Goal:** Build a simple password-strength meter and estimate entropy.



In [1]:
import string
import math

COMMON_PASSWORDS = {
    "password", "123456", "qwerty", "letmein", "pass123", "admin", "welcome"
}

def analyse_password_strength(pw: str):
    score = 0
    feedback = []

    length = len(pw)
    if length >= 8:
        score += 1
    else:
        feedback.append("Password is shorter than 8 characters.")
    if length >= 12:
        score += 1

    lowers = any(c.islower() for c in pw)
    uppers = any(c.isupper() for c in pw)
    digits = any(c.isdigit() for c in pw)
    symbols = any(c in string.punctuation for c in pw)

    categories = [lowers, uppers, digits, symbols]
    score += sum(1 for c in categories if c)

    if pw.lower() in COMMON_PASSWORDS:
        feedback.append("Password is in a list of very common passwords.")
        score = max(score - 2, 0)

    pool_size = 0
    if lowers: pool_size += 26
    if uppers: pool_size += 26
    if digits: pool_size += 10
    if symbols: pool_size += len(string.punctuation)
    entropy = length * math.log2(pool_size) if pool_size > 0 else 0

    if score <= 2:
        rating = "Very Weak"
    elif score == 3:
        rating = "Weak"
    elif score == 4:
        rating = "Medium"
    elif score == 5:
        rating = "Strong"
    else:
        rating = "Very Strong"

    return {
        "password": pw,
        "length": length,
        "score": score,
        "rating": rating,
        "entropy_bits": round(entropy, 2),
        "feedback": feedback
    }

for test_pw in ["Pass123", "MyP@ssw0rd!", "correcthorsebatterystaple"]:
    result = analyse_password_strength(test_pw)
    print("\nPassword:", result["password"])
    print("Rating:", result["rating"], "| Score:", result["score"], "| Entropy:", result["entropy_bits"], "bits")
    if result["feedback"]:
        print("Feedback:", "; ".join(result["feedback"]))



Password: Pass123
Rating: Very Weak | Score: 1 | Entropy: 41.68 bits
Feedback: Password is shorter than 8 characters.; Password is in a list of very common passwords.

Password: MyP@ssw0rd!
Rating: Strong | Score: 5 | Entropy: 72.1 bits

Password: correcthorsebatterystaple
Rating: Weak | Score: 3 | Entropy: 117.51 bits


---

##  Section 2 – Password Hashing (MD5, SHA-256, bcrypt)



In [2]:
import hashlib
import bcrypt

def hash_md5(pw: str) -> str:
    return hashlib.md5(pw.encode()).hexdigest()

def hash_sha256(pw: str) -> str:
    return hashlib.sha256(pw.encode()).hexdigest()

def hash_bcrypt(pw: str, rounds: int = 12) -> bytes:
    salt = bcrypt.gensalt(rounds)
    return bcrypt.hashpw(pw.encode(), salt)

def verify_bcrypt(pw: str, hashed: bytes) -> bool:
    return bcrypt.checkpw(pw.encode(), hashed)

test_password = "MyP@ssw0rd!"

print("MD5     :", hash_md5(test_password))
print("SHA-256 :", hash_sha256(test_password))

bcrypt_hash = hash_bcrypt(test_password)
print("bcrypt  :", bcrypt_hash.decode())
print("Verify OK:", verify_bcrypt(test_password, bcrypt_hash))


MD5     : 42c853d6bbd0cfddc2d0978df437fa97
SHA-256 : e493c394a28652900d73f0fc7e6713840b1af0ab1f3fd9c5878d82e5f753c6c1
bcrypt  : $2b$12$4HzYmhUnoMYvxUqpsDBUB.0r5NvvOMD9jCMFDVYhJLpNyqo1xW526
Verify OK: True


---

##  Section 3 – Salts and Peppers

**Goal:** Understand how salting and peppering protect against rainbow tables and database leaks.


In [3]:
import os

def simple_sha256(pw: str) -> str:
    return hashlib.sha256(pw.encode()).hexdigest()

def salted_sha256(pw: str, salt: bytes) -> str:
    return hashlib.sha256(salt + pw.encode()).hexdigest()

PEPPER = b"THIS_IS_A_SERVER_SIDE_SECRET_PEPPER"

def peppered_sha256(pw: str, salt: bytes, pepper: bytes = PEPPER) -> str:
    return hashlib.sha256(pepper + salt + pw.encode()).hexdigest()

password = "user123password"

print("=== Without Salt ===")
print(simple_sha256(password))
print(simple_sha256(password), "<= same password, same hash")

print("\n=== With Salt ===")
salt1 = os.urandom(16)
salt2 = os.urandom(16)
print("Hash 1:", salted_sha256(password, salt1))
print("Hash 2:", salted_sha256(password, salt2), "<= same password, different salts → different hashes")

print("\n=== With Salt + Pepper ===")
pep_hash1 = peppered_sha256(password, salt1)
pep_hash2 = peppered_sha256(password, salt2)
print("Peppered 1:", pep_hash1)
print("Peppered 2:", pep_hash2)


=== Without Salt ===
6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a <= same password, same hash

=== With Salt ===
Hash 1: d1a06e4388eb6fac9c6ec573e8140045e4b56fc6895328fd0a8e255314cf571f
Hash 2: 50d87711b514bc70610fef19e5ce26d390929ee2f9ce2828a62e6ebf7140fa62 <= same password, different salts → different hashes

=== With Salt + Pepper ===
Peppered 1: fd0e2f15f6b2d7f960e23aeffd50d11f7995c003c97c4c3d8bbd0b40ed622ecb
Peppered 2: 30febfc763d8181f3f4ddbaa09ebc28dd0f28aeabb0dd46a1cb4b4d45906a122


---

## Section 4 – TOTP (2FA) Implementation

**Goal:** Implement basic **Time-based One-Time Passwords (TOTP)** as a second factor.


In [4]:
import pyotp
import qrcode
from datetime import datetime

totp_secret = pyotp.random_base32()
print("TOTP Secret:", totp_secret)

totp = pyotp.TOTP(totp_secret)
uri = totp.provisioning_uri(name="student@example.com", issuer_name="NSS-Week03-Demo")
print("\nProvisioning URI:", uri)

img = qrcode.make(uri)
qr_path = "totp_qr.png"
img.save(qr_path)
print(f"✅ QR code saved as {qr_path} — scan it with an authenticator app.")

print("\nCurrent TOTP code:", totp.now())
print("Valid at time:", datetime.now())


TOTP Secret: JBFL2BL2HSF2ZDXRXOTPLPMEVL625XHH

Provisioning URI: otpauth://totp/NSS-Week03-Demo:student%40example.com?secret=JBFL2BL2HSF2ZDXRXOTPLPMEVL625XHH&issuer=NSS-Week03-Demo
✅ QR code saved as totp_qr.png — scan it with an authenticator app.

Current TOTP code: 959655
Valid at time: 2025-12-09 18:52:23.711569


---

##  Section 5 – Brute-Force Attack Simulation

**Goal:** Demonstrate how quickly weak hashes for common passwords can be cracked.


In [5]:
COMMON_PASSWORDS_LIST = [
    "password", "123456", "qwerty", "letmein", "iloveyou",
    "admin", "welcome", "monkey", "dragon", "football"
]

def brute_force_hash(target_hash: str, hash_type: str = "md5"):
    print(f"[*] Attempting {hash_type} brute-force against target hash:", target_hash)
    for pw in COMMON_PASSWORDS_LIST:
        if hash_type == "md5":
            h = hashlib.md5(pw.encode()).hexdigest()
        elif hash_type == "sha256":
            h = hashlib.sha256(pw.encode()).hexdigest()
        else:
            raise ValueError("Unsupported hash_type")

        if h == target_hash:
            print(f"[+] Match found! Password = '{pw}'")
            return pw
    print("[-] No match in common password list.")
    return None

target = "password"
md5_target = hashlib.md5(target.encode()).hexdigest()
sha_target = hashlib.sha256(target.encode()).hexdigest()

print("\n=== MD5 brute-force ===")
_ = brute_force_hash(md5_target, "md5")

print("\n=== SHA-256 brute-force ===")
_ = brute_force_hash(sha_target, "sha256")

print("\nNote: bcrypt is deliberately slow, so even a tiny dictionary attack would be much slower.")



=== MD5 brute-force ===
[*] Attempting md5 brute-force against target hash: 5f4dcc3b5aa765d61d8327deb882cf99
[+] Match found! Password = 'password'

=== SHA-256 brute-force ===
[*] Attempting sha256 brute-force against target hash: 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
[+] Match found! Password = 'password'

Note: bcrypt is deliberately slow, so even a tiny dictionary attack would be much slower.


---

##  Section 6 – Simple Authentication System

Combine:

- Password strength checks  
- bcrypt hashing  
- TOTP 2FA  

into a small, in-memory authentication system.


In [6]:
class AuthSystem:
    def __init__(self, min_entropy_bits: float = 40.0):
        self.users = {}  # username -> {'pw_hash': ..., 'totp_secret': ...}
        self.min_entropy_bits = min_entropy_bits

    def register_user(self, username: str, password: str):
        if username in self.users:
            raise ValueError("User already exists")

        analysis = analyse_password_strength(password)
        if analysis["entropy_bits"] < self.min_entropy_bits or analysis["rating"] in ("Very Weak", "Weak"):
            raise ValueError(f"Weak password. Rating: {analysis['rating']}, Entropy: {analysis['entropy_bits']} bits")

        pw_hash = hash_bcrypt(password)
        totp_secret = pyotp.random_base32()

        self.users[username] = {
            "pw_hash": pw_hash,
            "totp_secret": totp_secret
        }

        totp = pyotp.TOTP(totp_secret)
        uri = totp.provisioning_uri(name=username, issuer_name="NSS-Week03-System")
        return uri

    def authenticate(self, username: str, password: str, totp_code: str) -> bool:
        user = self.users.get(username)
        if not user:
            print("[-] User not found.")
            return False

        if not verify_bcrypt(password, user["pw_hash"]):
            print("[-] Invalid password.")
            return False

        totp = pyotp.TOTP(user["totp_secret"])
        if not totp.verify(totp_code):
            print("[-] Invalid or expired TOTP code.")
            return False

        print("[+] Authentication successful!")
        return True

# Demonstration (non-interactive)
auth = AuthSystem()

demo_user = "alice"
demo_password = "MySup3rStr0ngP@ss!"

print("\n=== Registering user ===")
uri = auth.register_user(demo_user, demo_password)
print("TOTP provisioning URI:")
print(uri)

demo_totp = pyotp.TOTP(auth.users[demo_user]["totp_secret"])
current_code = demo_totp.now()
print("\nCurrent TOTP code for demo:", current_code)

print("\n=== Authenticating user ===")
auth.authenticate(demo_user, demo_password, current_code)



=== Registering user ===
TOTP provisioning URI:
otpauth://totp/NSS-Week03-System:alice?secret=IN5SWXKHGINW5VJ2THW4DBVFXYOLBSGP&issuer=NSS-Week03-System

Current TOTP code for demo: 279591

=== Authenticating user ===
[+] Authentication successful!


True

---

##  Reflection
This lab helped me understand password strength beyond just “adding symbols.” I didn’t realise how much entropy and character variety affect how difficult a password is to brute-force. A password that looks complex can still be weak if it’s short or predictable.

I also saw why fast hashes like MD5 and SHA-256 are unsafe for storing passwords. Their speed makes large-scale guessing extremely easy for attackers. In contrast, bcrypt/Argon2 slow down each attempt, which massively increases the cost of brute-forcing a stolen hash.

Adding salt, pepper, and TOTP showed how multi-layered defence works. Even if an attacker gets the database, salts stop rainbow tables, peppers hide part of the secret, and 2FA blocks logins without the physical device.
To improve this system, I’d add rate limiting, account lockouts, better logging, and secure storage for secrets, which are all essential in real-world authentication systems.
